### Download data from drive

In [ ]:
import zipfile # Библиотека для работы с zip архивами
import os      # Библиотека для работы с фаловой системой 
import time    # Библиотека для работы со временем

from google.colab import drive # Модуль для работы с Google Disk
# Подключаем гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Прописываем путь к файлу с архивом
zip_file = '/content/drive/My Drive/bee/datasets.zip'  

# Распаковываем архив
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

# Просмотр результата разархивации
print(os.listdir())

['.config', 'drive', 'mobile_client.json', 'parent_operator.csv', 'agg_usage.parquet', 'web_client.json', 'sample_data']


### Install Kafka

In [ ]:
!pip install kafka-python

In [ ]:
import pandas as pd
import numpy as np
import threading
import json
from json import dumps
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [ ]:
!curl -sSOL https://downloads.apache.org/kafka/2.7.2/kafka_2.13-2.7.2.tgz
!tar -xzf kafka_2.13-2.7.2.tgz

In [ ]:
#используется для запуска Zookeeper c конфигами
!./kafka_2.13-2.7.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-2.7.2/config/zookeeper.properties

#используется для запуска Kafka с конфигами 
!./kafka_2.13-2.7.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-2.7.2/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


### Kafka - промежуточный топик

In [ ]:
# название топика transaction: разделы = 1, коэффициент репликации = 1 
!./kafka_2.13-2.7.2/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic transaction

Created topic transaction.


In [ ]:
# описание топика
!./kafka_2.13-2.7.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic transaction

Topic: transaction	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: transaction	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


##Заливаем JSON в kafka 

In [ ]:
# считываем сообщения из json последовательно

infile = open("web_client.json","r")

def get_next_message(infile):
  for line in infile: 
    new_dict = json.loads(line, next(infile))
  return new_dict

# получить сообщение = get_next_message(infile)

In [ ]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
                         value_serializer=lambda x:dumps(x).encode('utf-8'), 
                         compression_type='gzip')

my_topic = 'transaction'

In [ ]:
with open("web_client.json", "r") as f:
  for i, line in enumerate(f): 
    data = json.loads(line)
    if i > 50: # на небольшой части запросов
      break
    try:
        future = producer.send(topic = my_topic, value = data)
        #посмотри что такое timeout=10?
        record_metadata = future.get(timeout=10)
        
        print('--> The message has been sent to a topic: \
                {}, partition: {}, offset: {}' \
                .format(record_metadata.topic,
                    record_metadata.partition,
                    record_metadata.offset ))   
                                
    except Exception as e:
        print('--> It seems an Error occurred: {}'.format(e))

    finally:
        producer.flush()

--> The message has been sent to a topic:                 transaction, partition: 0, offset: 0
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 1
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 2
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 3
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 4
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 5
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 6
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 7
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 8
--> The message has been sent to a topic:                 transaction, partition: 0, offset: 9
--> The message has been sent to a topic:         

## Заливаем в Spark

In [ ]:
!pip install --force-reinstall pyspark==2.4.6

     |████████████████████████████████| 218.4 MB 36 kB/s 
     |████████████████████████████████| 197 kB 19.4 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.6-py2.py3-none-any.whl size=218814407 sha256=0639128a16b7ad82144653ee9412bb379ec0e6d1cdd3b686243889351f2729aa
  Stored in directory: /root/.cache/pip/wheels/f1/42/b0/ba397759613f4feb1611021a2503e60e344e546671b2ae04f8
Successfully built pyspark


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
#!pip install pyspark==2.4.8

--2021-11-27 18:02:01--  https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz’

spark-2.4.8-bin-had 100%[===================>] 224.97M  22.8MB/s    in 11s     

2021-11-27 18:02:13 (21.2 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz’ saved [235899716/235899716]

spark-2.4.8-bin-hadoop2.7/
spark-2.4.8-bin-hadoop2.7/LICENSE
spark-2.4.8-bin-hadoop2.7/NOTICE
spark-2.4.8-bin-hadoop2.7/R/
spark-2.4.8-bin-hadoop2.7/R/lib/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/DESCRIPTION
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Me

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2021-11-27 18:02:19--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’

spark-streaming-kaf 100%[===================>]  11.45M  --.-KB/s    in 0.1s    

2021-11-27 18:02:19 (110 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’ saved [12002039/12002039]



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time

kafka_topic_name = "transaction"
kafka_bootstrap_servers = 'localhost:9092'

In [ ]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,2)

kafka_topic_name = "transaction"
kafka_bootstrap_servers = 'localhost:9092'

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

lines = kvs.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(' '))
counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)
counts.pprint()
ssc.start()
# stream will run for 50 sec
ssc.awaitTerminationOrTimeout(50)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2021-11-27 18:04:02
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:04
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:06
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:08
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:10
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:12
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:14
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:16
-------------------------------------------

-------------------------------------------
Time: 2021-11-27 18:04:18
----------

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [ ]:
df = spark.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", "127.0.0.1:9092")\
.option("subscribe", "json_topic")\
.option("startingOffsets", "earliest")\
.load()

df.printSchema()

AnalysisException: ignored

In [ ]:
counts